### In this notebook, we will upload huggingface dataset to OLTP (Postgres in this context) and also use it as OLAP Bronze Zone (just for simple)

### Set up

In [1]:
import os
import sys

import datasets
import pandas as pd
import plotly.express as px
from datasets import load_dataset
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from sqlalchemy import create_engine

sys.path.insert(0, "..")

import src.visualization.setup
from src.utils.data_prep import parse_dt

load_dotenv("../.env")
datasets.logging.set_verbosity_error()

### Controller

In [ ]:
class Args(BaseModel):
    run_name: str = "000-prep-data"
    testing: bool = True
    notebook_persist_dp: str = None
    random_seed: int = 41

    train_fp: str = "../../data/train.parquet"
    val_fp: str = "../../data/val.parquet"

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    # Number of days left out not being pushed into the OLTP so that later we can simulate having them as new data
    num_days_holdout: int = 30
    holdout_fp: str = "../../data/holdout.parquet"

    # Output PostgreSQL table
    table_name: str = "amz_review_rating_raw"

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        if not self.testing:
            os.makedirs(self.notebook_persist_dp, exist_ok=True)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))